In [1]:
import openmm
print(openmm.version.full_version)

8.1.1.dev-ec797ac


In [2]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile

In [3]:
# 创建 PDBFixer 对象
fixer = PDBFixer(filename='3ugc.pdb')

# 找到缺失的残基
fixer.findMissingResidues()
print(fixer.missingResidues)

# 处理非标准残基
fixer.findNonstandardResidues()
print(fixer.nonstandardResidues)

# 将非标准残基转换为标准残基
fixer.replaceNonstandardResidues()

# 找到缺失的原子
fixer.findMissingAtoms()
print(fixer.missingAtoms)

# 添加缺失的原子
fixer.addMissingAtoms()

# 按照 pH 7.0 添加氢原子
fixer.addMissingHydrogens(7.0)

# 保存修复后的 PDB 文件
PDBFile.writeFile(fixer.topology, fixer.positions, open('fixed.pdb', 'w'))

{}
[]
{<Residue 14 (LYS) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (LYS)>, <Atom 4 (CD) of chain 0 residue 0 (LYS)>, <Atom 5 (CE) of chain 0 residue 0 (LYS)>, <Atom 6 (NZ) of chain 0 residue 0 (LYS)>], <Residue 29 (GLN) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (GLN)>, <Atom 4 (CD) of chain 0 residue 0 (GLN)>, <Atom 5 (OE1) of chain 0 residue 0 (GLN)>, <Atom 6 (NE2) of chain 0 residue 0 (GLN)>], <Residue 47 (GLU) of chain 0>: [<Atom 4 (CD) of chain 0 residue 0 (GLU)>, <Atom 5 (OE1) of chain 0 residue 0 (GLU)>, <Atom 6 (OE2) of chain 0 residue 0 (GLU)>], <Residue 99 (GLN) of chain 0>: [<Atom 4 (CD) of chain 0 residue 0 (GLN)>, <Atom 5 (OE1) of chain 0 residue 0 (GLN)>, <Atom 6 (NE2) of chain 0 residue 0 (GLN)>], <Residue 155 (THR) of chain 0>: [<Atom 3 (CG2) of chain 0 residue 0 (THR)>, <Atom 4 (OG1) of chain 0 residue 0 (THR)>], <Residue 159 (GLU) of chain 0>: [<Atom 3 (CG) of chain 0 residue 0 (GLU)>, <Atom 4 (CD) of chain 0 residue 0 (GLU)>, <Atom 5 (OE1) of chain 0 resid

In [4]:
# 读取修复后的 PDB 文件
pdb = PDBFile('fixed.pdb')

# 获取所有残基
residues = list(pdb.topology.residues())

# 打印所有残基
for residue in residues:
    print(residue)

<Residue 0 (GLN) of chain 0>
<Residue 1 (PHE) of chain 0>
<Residue 2 (GLU) of chain 0>
<Residue 3 (GLU) of chain 0>
<Residue 4 (ARG) of chain 0>
<Residue 5 (HIS) of chain 0>
<Residue 6 (LEU) of chain 0>
<Residue 7 (LYS) of chain 0>
<Residue 8 (PHE) of chain 0>
<Residue 9 (LEU) of chain 0>
<Residue 10 (GLN) of chain 0>
<Residue 11 (GLN) of chain 0>
<Residue 12 (LEU) of chain 0>
<Residue 13 (GLY) of chain 0>
<Residue 14 (LYS) of chain 0>
<Residue 15 (GLY) of chain 0>
<Residue 16 (ASN) of chain 0>
<Residue 17 (PHE) of chain 0>
<Residue 18 (GLY) of chain 0>
<Residue 19 (SER) of chain 0>
<Residue 20 (VAL) of chain 0>
<Residue 21 (GLU) of chain 0>
<Residue 22 (MET) of chain 0>
<Residue 23 (CYS) of chain 0>
<Residue 24 (ARG) of chain 0>
<Residue 25 (TYR) of chain 0>
<Residue 26 (ASP) of chain 0>
<Residue 27 (PRO) of chain 0>
<Residue 28 (LEU) of chain 0>
<Residue 29 (GLN) of chain 0>
<Residue 30 (ASP) of chain 0>
<Residue 31 (ASN) of chain 0>
<Residue 32 (THR) of chain 0>
<Residue 33 (GLY) of

In [5]:
from openmm.app import *
from openmm import *
from openmm.unit import *

# 读取蛋白 PDB 文件
pdb = PDBFile("fixed.pdb")

# 以 1.0 nm 的缓冲区添加 TIP3P 水
modeller = Modeller(pdb.topology, pdb.positions)

# 选择 AMBER 力场（支持蛋白和溶剂）
forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [6]:
from openmm.unit import molar, nanometer

# 计算所需的离子数目
atoms_list = list(modeller.topology.atoms())
num_waters = sum([1 for atom in atoms_list if atom.element.symbol == 'O'])

# 假设每个水分子的体积约为 0.0304 nm^3
water_volume_per_molecule = 0.0304 * nanometer**3

# 计算盒子的体积
box_volume = num_waters * water_volume_per_molecule

# 将 box_volume 转换为升
box_volume_in_liters = box_volume.value_in_unit(liter)

# 计算所需的离子数目
molar_to_num = lambda M, volume: int(M * volume * 6.022e23)  # 计算所需离子数

# 计算所需的NaCl和Glycine的粒子数
num_na = molar_to_num(1.8, box_volume_in_liters)  # 1.8 M NaCl
num_glycine = molar_to_num(0.1, box_volume_in_liters)  # 0.1 M Glycine

In [7]:
# 添加溶剂和离子
modeller.addSolvent(forcefield, model='tip3p', padding=1.0*nanometer, ionicStrength=1.8*molar)

# 添加Glycine
modeller.addSolvent(forcefield, model='tip3p', padding=1.0*nanometer, ionicStrength=0.1*molar)

# 保存添加溶剂和离子后的PDB文件
PDBFile.writeFile(modeller.topology, modeller.positions, open("solvated.pdb", "w"))

KeyboardInterrupt: 

In [8]:
# 构建系统
system = forcefield.createSystem(
    pdb.topology, 
    nonbondedMethod=PME,   # 采用 PME 处理静电
    nonbondedCutoff=1.0*nanometer, 
    constraints=HBonds     # 仅限制氢键，提高计算效率
)

In [9]:
from openmm.unit import kelvin, picosecond, picoseconds

integrator = LangevinIntegrator(
    298*kelvin,   # 目标温度
    1/picosecond, # 摩擦系数（heat bath coupling）
    0.002*picoseconds  # 积分步长 (2 fs)
)

In [10]:
# 设定计算平台（如果有 GPU，建议使用 CUDA）
platform = Platform.getPlatformByName("CUDA")

# 创建 Simulation 对象
simulation = Simulation(pdb.topology, system, integrator, platform)

# 加载蛋白初始坐标
simulation.context.setPositions(pdb.positions)

# 计算初始势能
state = simulation.context.getState(getEnergy=True)
print("intPotEng:", state.getPotentialEnergy())

intPotEng: 8252902298.736748 kJ/mol


In [11]:
from openmm import unit

# 正确的单位是 kilojoules_per_mole/nanometer
tolerance = 10 * unit.kilojoules_per_mole / unit.nanometer

# 调用 minimizeEnergy 函数
simulation.minimizeEnergy(tolerance=tolerance)

In [12]:
# 逐步升温，从 0K -> 298K
nsteps = 10000  # 10 ps 逐步升温
for temp in range(0, 301, 50):  # 每 50K 增加一次
    integrator.setTemperature(temp * kelvin)
    simulation.step(int(nsteps / 6))
    print(f"raising to {temp} K.")

raising to 0 K.
raising to 50 K.
raising to 100 K.
raising to 150 K.
raising to 200 K.
raising to 250 K.
raising to 300 K.


In [13]:
# NVT 平衡
print("NVT balancing...")
simulation.step(25000)  # 50 ps

NVT balancing...


In [14]:
# 添加恒压控制器 (1 atm)
system.addForce(MonteCarloBarostat(1*bar, 298*kelvin, 25)) 

# 运行 100 ps NPT 平衡
print("NPT balancing...")
simulation.step(25000)

NPT balancing...


In [15]:
# 设置输出文件
dcd_reporter = DCDReporter("output.dcd", 100)   # 每 1000 步存储一次轨迹
log_reporter = StateDataReporter("output.log", 100, step=True,
                                 potentialEnergy=True, temperature=True,
                                 density=True, progress=True,
                                 remainingTime=True, speed=True, totalSteps=50000)

# 添加 Reporter
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(log_reporter)

# 运行 0.01 ns 生产模拟（5,000 steps * 2 fs = 10 ns）
print("开始生产模拟...")
simulation.step(50000)

开始生产模拟...


In [5]:
def extract_atoms_in_box(pdb_file, center, size):
    center_x, center_y, center_z = center
    size_x, size_y, size_z = size
    
    # 计算包围盒范围
    x_min, x_max = center_x - size_x / 2, center_x + size_x / 2
    y_min, y_max = center_y - size_y / 2, center_y + size_y / 2
    z_min, z_max = center_z - size_z / 2, center_z + size_z / 2
    
    atoms_in_box = []

    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith(("ATOM")):  # 只处理原子数据
                x = float(line[30:38].strip())
                y = float(line[38:46].strip())
                z = float(line[46:54].strip())

                # 判断原子是否在指定范围内
                if x_min <= x <= x_max and y_min <= y <= y_max and z_min <= z <= z_max:
                    atoms_in_box.append(line.strip())  # 保存符合条件的原子行
    
    return atoms_in_box

In [29]:
# 示例输入
pdb_filename = "fixed.pdb"
center_coords_3 = (17.612, -13.059, -21.399)
box_size_3 = (64.0*0.375, 54.0*0.375, 74.0*0.375)
center_coords_2 = (17.612, -13.059, -26.737)
box_size_2 = (64.0*0.375, 54.0*0.375, 40.0*0.375)
center_coords_1 = (23.041, -7.401, -14.245)
box_size_1 = (40.0*0.375, 40.0*0.375, 40.0*0.375)

# 提取符合条件的原子
selected_atoms_3 = extract_atoms_in_box(pdb_filename, center_coords_3, box_size_3)
selected_atoms_2 = extract_atoms_in_box(pdb_filename, center_coords_2, box_size_2)
selected_atoms_1 = extract_atoms_in_box(pdb_filename, center_coords_1, box_size_1)

# 保存提取的原子
with open("selected_atoms_3.pdb", 'w') as f:
    f.write("\n".join(selected_atoms_3))

with open("selected_atoms_2.pdb", 'w') as f:
    f.write("\n".join(selected_atoms_2))

with open("selected_atoms_1.pdb", 'w') as f:
    f.write("\n".join(selected_atoms_1))

In [ ]:
# 读取 fixed.pdb 和 pocket1.pdb，pocket2.pdb，pocket3.pdb
with open('fixed.pdb', 'r') as fixed_file:
    fixed_lines = fixed_file.readlines()

with open('selected_atoms_1.pdb', 'r') as pocket1_file:
    pocket1_lines = pocket1_file.readlines()

with open('selected_atoms_2.pdb', 'r') as pocket2_file:
    pocket2_lines = pocket2_file.readlines()

with open('selected_atoms_3.pdb', 'r') as pocket3_file:
    pocket3_lines = pocket3_file.readlines()

# 将 pocket 的原子添加到 fixed.pdb 中
with open('combined.pdb', 'w') as combined_file:
    combined_file.writelines(fixed_lines)
    combined_file.writelines(pocket1_lines)
    combined_file.writelines(pocket2_lines)
    combined_file.writelines(pocket3_lines)

In [30]:
def check_atoms_in_fixed(fixed_file, selected_atoms_files):
    # 读取 fixed.pdb 文件中的所有原子
    with open(fixed_file, 'r') as f:
        fixed_atoms = set(line[12:26].strip() for line in f if line.startswith("ATOM"))
    
    for selected_file in selected_atoms_files:
        with open(selected_file, 'r') as f:
            selected_atoms = set(line[12:26].strip() for line in f if line.startswith("ATOM"))
        
        # 检查 selected_atoms 是否都在 fixed_atoms 中
        missing_atoms = selected_atoms - fixed_atoms
        if missing_atoms:
            print(f"以下原子在 {selected_file} 中，但不在 {fixed_file} 中:")
            for atom in missing_atoms:
                print(atom)
        else:
            print(f"{selected_file} 中的所有原子都在 {fixed_file} 中出现。")

# 示例调用
selected_files = ['selected_atoms_1.pdb', 'selected_atoms_2.pdb', 'selected_atoms_3.pdb']
check_atoms_in_fixed('fixed.pdb', selected_files)

selected_atoms_1.pdb 中的所有原子都在 fixed.pdb 中出现。
selected_atoms_2.pdb 中的所有原子都在 fixed.pdb 中出现。
selected_atoms_3.pdb 中的所有原子都在 fixed.pdb 中出现。


In [ ]:
from pymol import cmd

def color_pocket_atoms(fixed_obj, pocket_obj, color):
    # 获取 pocket.pdb 的所有原子信息
    pocket_atoms = cmd.get_model(pocket_obj).atom
    pocket_atom_set = set()

    # 记录 pocket 结构中的 (Atom ID, Atom Name)
    for atom in pocket_atoms:
        pocket_atom_set.add((atom.index, atom.name))  # index 即原子编号（Atom ID）

    # 遍历 fixed.pdb 的原子，匹配 pocket 结构的原子
    for atom_id, atom_name in pocket_atom_set:
        selection = f"{fixed_obj} and id {atom_id} and name {atom_name}"
        cmd.color(color, selection)  # 着色

# 加载 fixed 结构
cmd.load("fixed.pdb", "fixed_")

# 加载 pocket 结构
cmd.load("selected_atoms_1.pdb", "pocket1")
cmd.load("selected_atoms_2.pdb", "pocket2")
cmd.load("selected_atoms_3.pdb", "pocket3")

# 在 fixed.pdb 中匹配 pocket1, pocket2, pocket3 的原子并进行不同颜色的着色
color_pocket_atoms("fixed", "pocket1", "red")
color_pocket_atoms("fixed", "pocket2", "blue")
color_pocket_atoms("fixed", "pocket3", "green")

In [ ]:
from pymol import cmd

def color_pocket_atoms(fixed_obj, pocket_obj, color):
    # 获取 pocket.pdb 的所有原子信息
    pocket_atoms = cmd.get_model(pocket_obj).atom
    pocket_atom_set = set()

    # 记录 pocket 结构中的 (Atom ID, Atom Name)
    for atom in pocket_atoms:
        pocket_atom_set.add((atom.index, atom.name))  # index 即原子编号（Atom ID）

    # 遍历 fixed.pdb 的原子，匹配 pocket 结构的原子
    for atom_id, atom_name in pocket_atom_set:
        selection = f"{fixed_obj} and id {atom_id} and name {atom_name}"
        cmd.color(color, selection)  # 着色

# 加载 fixed 结构
cmd.load("fixed.pdb", "fixed_")

# 加载 pocket 结构
cmd.load("selected_atoms_1.pdb", "pocket1")

# 在 fixed.pdb 中匹配 pocket1, pocket2, pocket3 的原子并进行不同颜色的着色
color_pocket_atoms("fixed", "pocket1", "red")

In [ ]:
from pymol import cmd

def color_pocket_atoms_by_id_and_name(fixed_obj, pocket_obj, color):
    pocket_atoms = cmd.get_model(pocket_obj).atom  # 获取 pocket.pdb 的原子信息
    
    for atom in pocket_atoms:
        atom_id = atom.index  # 获取原子 ID
        atom_name = atom.name  # 获取原子名称 (O, CA, N, etc.)

        # 在 fixed 结构中找到相应原子
        selection = f"{fixed_obj} and id {atom_id} and name {atom_name}"
        cmd.color(color, selection)

# 加载结构
cmd.load("fixed.pdb", "fixed_")
cmd.load("selected_atoms_1.pdb", "pocket1")

# 颜色标记
color_pocket_atoms_by_id_and_name("fixed", "pocket1", "red")

In [ ]:
from pymol import cmd

def color_pocket_atoms_by_id_and_name(fixed_obj, pocket_obj, color):
    pocket_atoms = cmd.get_model(pocket_obj).atom  # 获取 pocket.pdb 的原子信息
    
    for atom in pocket_atoms:
        atom_id = atom.index  # 获取原子 ID

        # 在 fixed 结构中找到相应原子
        selection = f"{fixed_obj} and id {atom_id}"
        print(selection)
        cmd.color(color, selection)

# 加载结构
cmd.load("fixed.pdb", "fixed_")
cmd.load("selected_atoms_1.pdb", "pocket1")

# 颜色标记
cmd.color("grey50", "fixed_")
color_pocket_atoms_by_id_and_name("fixed_", "pocket1", "red")

fixed_ and id 1
fixed_ and id 2
fixed_ and id 3
fixed_ and id 4
fixed_ and id 5
fixed_ and id 6
fixed_ and id 7
fixed_ and id 8
fixed_ and id 9
fixed_ and id 10
fixed_ and id 11
fixed_ and id 12
fixed_ and id 13
fixed_ and id 14
fixed_ and id 15
fixed_ and id 16
fixed_ and id 17
fixed_ and id 18
fixed_ and id 19
fixed_ and id 20
fixed_ and id 21
fixed_ and id 22
fixed_ and id 23
fixed_ and id 24
fixed_ and id 25
fixed_ and id 26
fixed_ and id 27
fixed_ and id 28
fixed_ and id 29
fixed_ and id 30
fixed_ and id 31
fixed_ and id 32
fixed_ and id 33
fixed_ and id 34
fixed_ and id 35
fixed_ and id 36
fixed_ and id 37
fixed_ and id 38
fixed_ and id 39
fixed_ and id 40
fixed_ and id 41
fixed_ and id 42
fixed_ and id 43
fixed_ and id 44
fixed_ and id 45
fixed_ and id 46
fixed_ and id 47
fixed_ and id 48
fixed_ and id 49
fixed_ and id 50
fixed_ and id 51
fixed_ and id 52
fixed_ and id 53
fixed_ and id 54
fixed_ and id 55
fixed_ and id 56
fixed_ and id 57
fixed_ and id 58
fixed_ and id 59
fixed_

In [ ]:
from pymol import cmd

def color_pocket_atoms_by_id_and_name(fixed_obj, pocket_obj, color):
    pocket_atoms = cmd.get_model(pocket_obj).atom  # 获取 pocket.pdb 的原子信息
    
    for atom in pocket_atoms:
        atom_id = atom.index  # 获取原子 ID

        # 在 fixed 结构中找到相应原子
        selection = f"{fixed_obj} and id {atom_id}"
        print(selection)
        cmd.color(color, selection)

# 加载结构
cmd.load("fixed.pdb", "fixed_")
cmd.load("selected_atoms_2.pdb", "pocket2")

# 颜色标记
cmd.color("grey50", "fixed_")
color_pocket_atoms_by_id_and_name("fixed_", "pocket2", "green")

In [ ]:
from pymol import cmd

def color_pocket_atoms_by_id_and_name(fixed_obj, pocket_obj, color):
    pocket_atoms = cmd.get_model(pocket_obj).atom  # 获取 pocket.pdb 的原子信息
    
    for atom in pocket_atoms:
        atom_id = atom.index  # 获取原子 ID

        # 在 fixed 结构中找到相应原子
        selection = f"{fixed_obj} and id {atom_id}"
        print(selection)
        cmd.color(color, selection)

# 加载结构
cmd.load("fixed.pdb", "fixed_")
cmd.load("selected_atoms_3.pdb", "pocket3")

# 颜色标记
cmd.color("grey50", "fixed_")
color_pocket_atoms_by_id_and_name("fixed_", "pocket3", "blue")